In [1]:
%load_ext autotime
import geopandas as gpd
import pandas as pd
import numpy as np
from scipy.spatial import KDTree
from tqdm.auto import tqdm
from glob import glob
from shapely.geometry import LineString, Point
import folium
pd.set_option('display.max_columns', None)

In [2]:
new_shorelines = gpd.GeoDataFrame.from_features([{"type":"Feature","properties":{},"geometry":{"type":"LineString","coordinates":[[172.685782,-43.630577],[172.68891,-43.633128]]}}])
new_shorelines.crs = 4326
new_shorelines

,geometry
0,"LINESTRING (172.68578 -43.63058, 172.68891 -43..."


In [3]:
m = new_shorelines.explore(tiles="ESRI.WorldImagery")
gpd.GeoSeries(new_shorelines.geometry.apply(lambda line: Point(line.coords[0])), crs=new_shorelines.crs).explore(m=m, color="red", name="transect start")
m

In [4]:
shorelines = gpd.read_file("shorelines.geojson")
transects = gpd.read_file("transects_extended.geojson")
poly = gpd.read_file("polygons.geojson")

In [5]:
nz_shorelines = shorelines[shorelines.id.str.startswith("nzd")]
latest_siteid = nz_shorelines.id.max()
latest_siteid_int = int(latest_siteid[4:])
new_siteids = new_shorelines.index.to_series().apply(lambda i: f"nzd0{latest_siteid_int + i + 1}")
print(f"Latest siteid is {latest_siteid}, so the new sites will be {new_siteids}")
new_shorelines["id"] = new_siteids
new_shorelines

Latest siteid is nzd0580, so the new sites will be 0    nzd0581
dtype: object


,geometry,id
0,"LINESTRING (172.68578 -43.63058, 172.68891 -43...",nzd0581


In [6]:
pd.concat((shorelines, new_shorelines)).to_file("shorelines.geojson", driver="GeoJSON")

In [7]:
new_polys = gpd.GeoDataFrame({"id": new_siteids}, geometry=new_shorelines.to_crs(2193).buffer(100).minimum_rotated_rectangle().to_crs(4326), crs=4326)
new_polys.explore()

In [8]:
pd.concat((poly, new_polys)).to_file("polygons.geojson", driver="GeoJSON")

In [9]:
nz_transects = transects[transects.id.str.startswith("nzd")].to_crs(2193)
nz_transects

,id,site_id,orientation,along_dist,along_dist_norm,beach_slope,cil,ciu,trend,n_points,n_points_nonan,r2_score,mae,mse,rmse,intercept,ERODIBILITY,geometry
95050,nzd0001-0000,nzd0001,359.037136,3197.737936,1.000000,0.110,0.0681,0.2000,-0.174010,196.0,192.0,0.003917,17.659820,458.983904,21.423910,325.937325,None,"LINESTRING (1596662.375 6190263.089, 1596646.6..."
95051,nzd0001-0001,nzd0001,359.037136,3097.737936,0.968728,0.115,0.0738,0.2000,-0.181775,196.0,193.0,0.005612,15.338270,353.476544,18.800972,328.697573,None,"LINESTRING (1596744.903 6190264.499, 1596729.1..."
95052,nzd0001-0002,nzd0001,359.037136,2997.737936,0.937456,0.120,0.0770,0.2000,-0.306998,196.0,193.0,0.019757,14.242079,282.327917,16.802616,334.822195,None,"LINESTRING (1596827.431 6190265.908, 1596811.6..."
95053,nzd0001-0003,nzd0001,359.037136,2897.737936,0.906184,0.065,0.0546,0.0759,-0.344997,196.0,194.0,0.032463,12.443181,217.289816,14.740754,344.022130,None,"LINESTRING (1596909.959 6190267.317, 1596894.2..."
95054,nzd0001-0004,nzd0001,359.037136,2797.737936,0.874912,0.065,0.0568,0.0788,-0.418404,196.0,194.0,0.052214,11.504372,194.647497,13.951613,356.546817,None,"LINESTRING (1596992.486 6190268.724, 1596976.7..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147486,nzd0580-0005,nzd0580,NaN,NaN,NaN,0.130,0.1009,0.1853,-0.029925,685.0,684.0,0.001042,5.960526,48.393340,6.956532,217.218482,None,"LINESTRING (1575247.391 5169328.227, 1575202.3..."
147487,nzd0580-0006,nzd0580,NaN,NaN,NaN,0.045,0.0387,0.0536,0.122706,685.0,684.0,0.004371,11.345870,192.259499,13.865767,177.311309,None,"LINESTRING (1575055.33 5169296.88, 1575242.365..."
147488,nzd0580-0007,nzd0580,NaN,NaN,NaN,0.025,0.0240,0.0254,-0.204064,685.0,684.0,0.040748,5.793930,54.947949,7.412688,207.907319,None,"LINESTRING (1574984.641 5169259.468, 1575171.6..."
147489,nzd0580-0008,nzd0580,NaN,NaN,NaN,0.200,0.0710,0.2000,2.609341,685.0,681.0,0.135278,29.167891,2431.964139,49.314948,620.170970,None,"LINESTRING (1574855.793 5168989.166, 1575250.0..."


In [11]:
def create_transects(line, spacing=80, transect_length=400):
    transects = []
    distances = np.arange(0, line.length, spacing)

    for distance in distances:
        # Find point along the average line
        point = line.interpolate(distance)
        
        # Find the direction of the line at this point (tangent direction)
        nearest_point_ahead = line.interpolate(min(distance + 1e-6, line.length))
        direction = np.arctan2(nearest_point_ahead.y - point.y, nearest_point_ahead.x - point.x)
        
        # Rotate 90 degrees (perpendicular) and extend to create a transect
        transect = LineString(reversed([
            Point(
                point.x - transect_length / 2 * np.cos(direction + np.pi / 2),
                point.y - transect_length / 2 * np.sin(direction + np.pi / 2)
            ),
            Point(
                point.x + transect_length / 2 * np.cos(direction + np.pi / 2),
                point.y + transect_length / 2 * np.sin(direction + np.pi / 2)
            )
        ]))
        
        transects.append(transect)
    
    transects = gpd.GeoDataFrame(geometry=transects, crs=2193).to_crs(4326)
    return transects

all_new_transects = []
for new_shoreline in new_shorelines.to_crs(2193).itertuples():
    new_transects = create_transects(new_shoreline.geometry)
    new_transects["id"] = new_shoreline.id + "-" + new_transects.index.astype(str).str.pad(4, fillchar="0")
    new_transects["site_id"] = new_shoreline.id
    all_new_transects.append(new_transects)

all_new_transects = gpd.GeoDataFrame(pd.concat(all_new_transects).reset_index(drop=True), crs=4326)
display(all_new_transects)
m = all_new_transects.explore()
new_shorelines.explore(m=m)
new_polys.boundary.explore(m=m)
gpd.GeoSeries(all_new_transects.geometry.apply(lambda line: Point(line.coords[0])), crs=all_new_transects.crs).explore(m=m, color="red", name="transect start")
print("Make sure the origin is inland")
m

,geometry,id,site_id
0,"LINESTRING (172.68763 -43.62938, 172.68393 -43...",nzd0581-0000,nzd0581
1,"LINESTRING (172.68829 -43.62992, 172.68459 -43...",nzd0581-0001,nzd0581
2,"LINESTRING (172.68895 -43.63046, 172.68525 -43...",nzd0581-0002,nzd0581
3,"LINESTRING (172.68961 -43.63099, 172.68591 -43...",nzd0581-0003,nzd0581
4,"LINESTRING (172.69027 -43.63153, 172.68657 -43...",nzd0581-0004,nzd0581


Make sure the origin is inland


In [12]:
pd.concat((transects, all_new_transects)).to_file("transects_extended.geojson", driver="GeoJSON")